In [5]:
] activate

 Activating environment at `/tank/home/ciaran/Dropbox/EMAC/DCSideBatteryModeling/Project.toml`


In [6]:
using LinearAlgebra
using ElectricalEngineering
using Interpolations

using OrdinaryDiffEq #Gets the solvers
using Plots
using PowerSystems
using DelimitedFiles

include(joinpath(pwd(), "DCSideBatteryModeling", "DCSideBatteryModeling.jl"))

In [7]:
plotly()

Plots.PlotlyBackend()

In [8]:
include(joinpath(pwd(), "data","make_data.jl"))

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/pm_io/common.jl:26
┌ Info: Constructing System from Power Models
│   data["name"] = omib
│   data["source_type"] = pti
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:39
┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /home/ciaran/.julia/packages/InfrastructureSystems/WC1sp/src/in_memory_time_series_storage.jl:24
┌ Info: Reading bus data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:140
┌ Info: Reading generator data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:390
┌ Info: Reading branch data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:543
┌ Info: Reading branch data
└ @

LoadError: UndefKeywordError: keyword argument basepower not assigned

In [9]:
omib_sys = System(joinpath(pwd(), "data", "OMIB_inverterDCside.json"))

ErrorException: type NamedTuple has no field validation_descriptor_file

In [10]:
parameter_mapping = instantiate_parameters(omib_sys)
M= instantiate_4th_order_model(omib_sys)
u0 = M(parameter_mapping)

25-element Array{Float64,1}:
  1.0067552230867673
 -0.04029913889433856
  0.20330299948998976
  0.05885597151382373
  0.20030310784149463
 -0.01608748059654234
  0.2023045116349568
  0.008124112353003959
  0.00027215096174116113
 -2.185798994095428e-5
  0.07044511413183478
 -0.002805774194391405
  1.1
  0.30318458195205195
  0.30318458195205195
  0.30318458195205195
  0.000514025509223178
  3.1135906395677085e-9
  3.122546299733612e-18
  0.038091160415475785
 -8.428913687045097e-30
  3.2052620336590135e-9
 -1.2621774530555963e-29
  0.0
  1.0016443855184418e-13

In [11]:
jac_exp = get_4th_order_jacobian_expression()
_jac = eval(jac_exp)

#85 (generic function with 1 method)

In [12]:
jac_eval = (out, u0, params) -> _jac(out, u0, params)
param_eval = (out, params) -> _jac(out, M.u0, params)
n = length(M.u0)
J = zeros(n, n);

In [13]:
_parameter_values = [x.second for x in parameter_mapping]
M(_parameter_values)
param_eval(J, _parameter_values)
jac = ModelJacobian(jac_eval, J)
eigvals(jac(M))

25-element Array{Complex{Float64},1}:
 -2.7143807142475285e6 + 0.0im
    -26667.03419535636 + 0.0im
    -12741.51372613847 - 11476.015411109347im
    -12741.51372613847 + 11476.015411109347im
    -11875.53390126334 + 0.0im
    -9931.198785405673 - 4785.655096737608im
    -9931.198785405673 + 4785.655096737608im
     -9009.26332310856 - 418.3967308378123im
     -9009.26332310856 + 418.3967308378123im
   -3055.4174814740663 - 6375.078163796365im
   -3055.4174814740663 + 6375.078163796365im
   -2875.3510302853983 - 6053.534154191113im
   -2875.3510302853983 + 6053.534154191113im
    -2792.727251917347 + 0.0im
    -696.3859675196234 + 0.0im
    -491.0674322412798 - 18.545982773306342im
    -491.0674322412798 + 18.545982773306342im
   -186.31287113149455 - 753.4364875384816im
   -186.31287113149455 + 753.4364875384816im
    -37.70839358295397 + 0.0im
   -37.688358199235914 + 0.0im
   -11.400157563307424 - 0.005763462560475282im
   -11.400157563307424 + 0.005763462560475282im
     -4.8718571

In [17]:
param_range = 0.25:0.25:10
gains_1mF = reshape([],0,4)
_parameter_values[44]=0
for kpv in param_range
    println(kpv)
    _parameter_values[40]=kpv
    for kiv in param_range
        _parameter_values[41]=kiv
        for kpi in param_range
            _parameter_values[42]=kpi
            for kii in param_range
                _parameter_values[43]=kii
                M(_parameter_values)
                param_eval(J, _parameter_values)
                jac = ModelJacobian(jac_eval, J)
                eig_vals = eigvals(jac(M))
                damp = [-1*real(λ) / sqrt(real(λ)^2 + imag(λ)^2) for λ in eig_vals]
                if maximum(real(eig_vals)) < -3 && minimum(damp)>0.35
                    gains_1mF = [gains_1mF; [kpv kiv kpi kii ]]
                end
            end 
            
        end
        
    end
    
end

0.25
0.5
0.75
1.0
1.25
1.5
1.75
2.0
2.25
2.5
2.75
3.0
3.25
3.5
3.75
4.0
4.25
4.5
4.75
5.0
5.25
5.5
5.75
6.0
6.25
6.5
6.75
7.0
7.25
7.5
7.75
8.0
8.25
8.5
8.75
9.0
9.25
9.5
9.75
10.0


In [18]:
gains_1mF

26050×4 Array{Any,2}:
 0.25   2.0   1.75  10.0
 0.25   2.0   2.0    9.5
 0.25   2.0   2.0    9.75
 0.25   2.0   2.0   10.0
 0.25   2.0   2.25   9.0
 0.25   2.0   2.25   9.25
 0.25   2.0   2.25   9.5
 0.25   2.0   2.25   9.75
 0.25   2.0   2.25  10.0
 0.25   2.25  1.25   9.75
 0.25   2.25  1.25  10.0
 0.25   2.25  1.5    9.0
 0.25   2.25  1.5    9.25
 ⋮                  
 1.5   10.0   2.0    7.25
 1.5   10.0   2.0    7.5
 1.5   10.0   2.0    7.75
 1.5   10.0   2.0    8.0
 1.5   10.0   2.0    8.25
 1.5   10.0   2.0    8.5
 1.5   10.0   2.0    8.75
 1.5   10.0   2.0    9.0
 1.5   10.0   2.0    9.25
 1.5   10.0   2.0    9.5
 1.5   10.0   2.0    9.75
 1.5   10.0   2.0   10.0

In [19]:
writedlm( "results/stable_gains_1mF.csv",  gains_1mF, ',')

gains_1mF = readdlm("stable_gains_1mF.csv", ',', Float64)

In [ ]:
optIndex = readdlm( "results/optIndex.csv", ',')

In [23]:
optIndex = [1, 1e6]

2-element Array{Float64,1}:
 1.0
 1.0e6

In [ ]:
_parameter_values[44]=0 #K_pred
for i in 1:1:10000
    println(i)
    index = rand(1:size(gains_1mF,1))
    _parameter_values[40]=gains_1mF[index,1] #K_p^v
    _parameter_values[41]=gains_1mF[index,2] #K_i^v
    _parameter_values[42]=gains_1mF[index,3] #K_p^i
    _parameter_values[43]=gains_1mF[index,4] #K_i^i
    M(_parameter_values)

    disturbance=deepcopy(_parameter_values)
    disturbance[9]=0.7

    sim = instantiate_4th_order_dae(disturbance, M; tspan = (0.0, 0.02))
    sol = solve(sim, Rodas5(), saveat = 0.001);
    solArray = Array(sol)
    global runError=norm(_parameter_values[39].-solArray[13,:])
    if runError < optIndex[2]
        optIndex[1]= index
        optIndex[2]=runError
        writedlm( "results/optIndex.csv",  optIndex, ',')
        println(optIndex)
    end
end

1
[16165.0, 0.09611444580567789]
2
3
4
5
6
7
8
9
10
11
12
13
14
[18745.0, 0.08123332200452046]
15
16
17
[23451.0, 0.07649260738013298]
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
[22109.0, 0.07647201235152337]
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
